<a href="https://colab.research.google.com/github/dclark18/7331DataMiningNotebooks/blob/master/ICA3_DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (double click to edit):

- Name 1:Daniel Clark
- Name 2:Armando Vela
- Name 3:Joe Schueder
- Name 4: Jeff Washburn

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [1]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np
from __future__ import print_function

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.121435315436 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset? ***11,314 observations. The 130,107 represents all the unique words in the document.***
- What does each instance represent? ***Newsgroups corresponding to different topics***
- How many classes are in the dataset and what does each class represent? ***130,108 classe***
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not? ***How old is the 20 news groups dataset. Would we generalize the documents written in the past week. Also could be 1995.***
- Is the data represented as a sparse or dense matrix? ***Sparse Matrix. each individual word is only used in a handful of documents. Stores infrequently used items in the background and frequent items as their own individual rows.***

___
Enter your answer here:

- How many instances are in the dataset? 11314  

- What does each instance represent? Documents 

- How many classes are in the dataset and what does each class represent? The 20 Newsgroups data set. The 20 Newsgroups data set is a collection of approximately 20,000 newsgroup documents,  partitioned (nearly) evenly across 20 different newsgroups. To the best of my knowledge, it was originally collected by Ken Lang, probably for his Newsweeder: Learning to filter netnews paper, though he does not explicitly mention this collection. The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering. 

Each subdirectory in the bundle represents a newsgroup; each file in a subdirectory is the text of some newsgroup document that was posted to that newsgroup.   

-  How many classes are in the dataset and what does each class represent? Each class represents a newsgroup. There are 20 Classes. alt.atheism comp.graphics comp.os.ms-windows.misc comp.sys.ibm.pc.hardware comp.sys.ibm.pc.hardware comp.sys.mac.hardware comp.windows.x misc.forsale rec.autos rec.motorcycles rec.sport.baseball rec.sport.hockey sci.crypt sci.electronics sci.med sci.space soc.religion.christian talk.politics.guns talk.politics.mideast talk.politics.misc talk.religion.misc 

- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not? The dataset is from 1995, which is 25 years ago. Though many of the high level topics are topics that are discussed today jargon, slang and technology has changed significantly since 1995. I think it may work ok on less technological topics, but not very well at all related to technology.  For example the word corona would have a different meaning than it did in 1995 

- Is the data represented as a sparse or dense matrix? The ds command states that it is a sparse matrix. {'data': <11314x130107 sparse matrix of type '<class 'numpy.float64'>' with 1787565 stored elements in Compressed Sparse Row format>,

___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [7]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

# Enter distance comparison below for each pair of vectors:
p = 'Placeholder'
print('\n\nEuclidean Distance\n ab:', euclidean(a,b), 'ac:', euclidean(a,c), 'bc:',euclidean(b,c))
print('Cosine Distance\n ab:', cosine(a,b), 'ac:', cosine(a,c), 'bc:', cosine(b,c))
print('Jaccard Dissimilarity (vectors should be boolean values)\n ab:', jaccard(a>0,b>0), 'ac:', jaccard(a>0,c>0), 'bc:', jaccard(b>0,c>0))

print('The most appropriate distance is Cosine.Because the distances generated from this method make most sense. Bikes and Autos are more related, and Computers are the same distance from Bikes/autos.That is visible from Cosine distance results.')
print('Because the distances generated from this method make most sense. Bikes and Autos are more related, and Computers are the same distance from Bikes/autos.That is visible from Cosine distance results.') 

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 1.09851846719 ac: 1.18914054254 bc: 0.917779422666
Cosine Distance
 ab: 0.6033714113755322 ac: 0.7070276149559529 bc: 0.4211595343347173
Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.8821138211382114 ac: 0.8754716981132076 bc: 0.9087947882736156
The most appropriate distance is Cosine.Because the distances generated from this method make most sense. Bikes and Autos are more related, and Computers are the same distance from Bikes/autos.That is visible from Cosine distance results.
Because the distances generated from this method make most sense. Bikes and Autos are more related, and Computers are the same distance from Bikes/autos.That is visible from Cosine distance results.


___
# Start of Live Session Assignment
___
<a id="KNN"></a>
<a href="#top">Back to Top</a>
## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* 

**Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [13]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets 

cv = StratifiedShuffleSplit(test_size = 0.5, train_size=0.5, random_state=12345)

# fill in the training and testing data and save as separate variables
for trainidx, testidx in cv.split(ds.data,ds.target):
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K
acc_dict = {}
for K in range(1, 11):
  clf = KNeighborsClassifier(n_neighbors=K, weights='uniform', metric='euclidean')
  clf.fit(X_train, y_train)
  yhat = clf.predict(X_test)
  print('Accuracy of classifier with %d neighbors is: %.2f'%(K,accuracy_score(y_test, yhat)))

  acc_dict.update( {K : accuracy_score(y_test, yhat)} )


maximum = max(acc_dict, key=acc_dict.get)
acc = acc_dict[maximum] # overwrite this with the actual accuracy

#=====================================

print('The maximum accuracy of the classifier is %.2f with %d neighbors'%(acc,maximum))


Accuracy of classifier with 1 neighbors is: 0.60
Accuracy of classifier with 2 neighbors is: 0.56
Accuracy of classifier with 3 neighbors is: 0.53
Accuracy of classifier with 4 neighbors is: 0.51
Accuracy of classifier with 5 neighbors is: 0.49
Accuracy of classifier with 6 neighbors is: 0.48
Accuracy of classifier with 7 neighbors is: 0.47
Accuracy of classifier with 8 neighbors is: 0.47
Accuracy of classifier with 9 neighbors is: 0.46
Accuracy of classifier with 10 neighbors is: 0.45
The maximum accuracy of the classifier is 0.60 with 1 neighbors


The best accuracy is 0.60 with 1 neighbor.

**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

No. The use of a KD Tree does not make sense when you are working with Sparse Data.  The cost of Creating a Tree is going to be much greater than that of building a tree using KNN when there is text involved and the number of classifiers. With KNN, measuring distance is relatively easy on a computer, while building a tree will be difficult to branch since it's highly binary and is going to have very many zeros. 
___

_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. 

**Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [19]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

# the parameters for the nearest centroid metric to test are:
#    l1, l2, and cosine (all are optimized)
#•	Use a for loop to test your different distance metrics
#	for d in ['l1','l2','cosine']:
#	clf = NearestCentroid(metric=d)  
for distance in ['l1', 'cosine', 'l2']:
  clf = NearestCentroid(metric=distance)
  clf.fit(X_train, y_train)
  yhat = clf.predict(X_test)
  acc = accuracy_score(y_test, yhat)

  print(distance, acc)

# fill in your code here


print('The best distance metric is: Cosine since it is the best accuracy')

l1 0.30245713275587766
cosine 0.4811737670143185
l2 0.4132932649814389
The best distance metric is: Cosine since it is the best accuracy


___
<a id="naive"></a>
<a href="#top">Back to Top</a>
## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions: 

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

Since we have 130,107 features along with 20 classes, we will need to multipy the two to get 2,602,140 conditional probabilities. There is one prior for each class, to make a total of 20 priors which must be parameterized. 

___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: 

**Questions for part 6**: 
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
- What if the data is sparse, does this change your answer? Why or Why not?

Enter you answer here:

1. ) For continuous attributes, Gaussian Naive Bayes as it normalizes continuous data to fit a naive Bayes format, which minimizes the effect of dependence. That said, it would assume that the data is normally distributed on each feature. 
Multinomial and Binaries are better suited for binary and nominal/ordinal classes, and not appropriate for continuous data. 

2.) In dealing with sparse data, we would not want to use a Gaussian Naive Bayes, and would rather use Multinomial Naive Bayes instead, as it's better suited for text data. Sparse matrices are very difficult to capture realistic Gaussian models since the mean is often close to zero. 

You could also potentially binarize our text classification data for a specific model, and in that case a Bernoulli Naive Bayes will work. 


___

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). 

**Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. 

**Question for part 7**: Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)? 

In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

a_list = [0.0, 0.001, 0.01, 0.1, 1]
b_list = [0.0, 0.002, 0.02, 0.04, 0.06, 0.08, 0.1]

for a in a_list:
    clf_mnb = MultinomialNB(alpha= a)
    acc = clf_mnb.fit(X_train, y_train).score(X_test, y_test)
    print('Accuracy of MultinomialNB classifier for alpha = %.3f is: %.3f'%(a,acc))
    

for a in a_list:
    print ('='*80)
    for b in b_list:
        clf_bnb = BernoulliNB(alpha=a, binarize=b)
        acc = clf_bnb.fit(X_train, y_train).score(X_test, y_test)
        print("Accuracy of BernoulliNB classifier for (alpha=%.3f, binarize=%.3f) is: %.3f" % (a, b, acc))



Accuracy of MultinomialNB classifier for alpha = 0.000 is: 0.869
Accuracy of MultinomialNB classifier for alpha = 0.001 is: 0.893
Accuracy of MultinomialNB classifier for alpha = 0.010 is: 0.887
Accuracy of MultinomialNB classifier for alpha = 0.100 is: 0.833
Accuracy of MultinomialNB classifier for alpha = 1.000 is: 0.718
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.000) is: 0.856
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.002) is: 0.857
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.020) is: 0.870
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.040) is: 0.847
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.060) is: 0.791
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.080) is: 0.741
Accuracy of BernoulliNB classifier for (alpha=0.000, binarize=0.100) is: 0.680
Accuracy of BernoulliNB classifier for (alpha=0.001, binarize=0.000) is: 0.841
Accuracy of BernoulliNB classifier for (alp

Within the Multinomial classifier, the top alpha score we found was alpha 0.001, which gave us an accuracy of 0.893. 

Within Bernouli's Classifier, the top binarize score was 0.020 across all the alpha selections we chose. In addition, as long as our alpha is below, 0.1, we can get a high accuracy. 

The reason they are able to perform so quickly is that the model is built on counting and dividing to create probabilities. In addition, naive bayes assumes independence which decouples the features from one another without the need to account for many weights. Also there is no iteration, there is no epoch, and the alpha values helps to smoothen the model and reduce the impact of features that don't play a role in model performance, and reduces computation required. 

________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.